### Calculating Null Distribution



Null distribution - is generated by getting the median correlation score of randomly combined replicates that do not come from the same compounds.



### The goal here: 

-- is to compute the **p-value** for each compound per dose by evaluating the probability of random combinations of replicates (from different compounds) having greater median correlation score than replicates that come from the same compound.




- In our case, we generated 1000 median correlation scores from randomly combined replicates as the **null distribution** for each CPD_SIZE class per DOSE i.e. for a cpd_size class for every DOSE (1-6) - we have 1000 medians scores from randomly combined replicates of different compounds.







Cpd_size is the number of replicates in a specific compound and cpd_size class is a specific group of compounds that have the same amount of replicates e.g all compounds with 3 replicates in them are in the same cpd_size class.

In [1]:
import os
import requests
import pickle
import pandas as pd
import numpy as np
import re
from os import walk
from collections import Counter
import random
import shutil
from statistics import median
import cmapPy.pandasGEXpress.parse_gct as pg
from cmapPy.pandasGEXpress.parse import parse

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

### - Load in Level 4 Datasets generated from `calculate_median_scores_notebook`

In [2]:
L1000_level4_path = "L1000_lvl4_cpd_replicate_datasets"

In [3]:
df_level4 = pd.read_csv(os.path.join(L1000_level4_path, 'L1000_level4W_cpd_replicates.csv.gz'), 
                        compression='gzip',low_memory = False)
df_cpd_med_scores = pd.read_csv(os.path.join(L1000_level4_path, 'cpd_replicate_median_scores_W.csv'))

In [4]:
df_cpd_med_scores = df_cpd_med_scores.set_index('cpd').rename_axis(None, axis=0).copy()

In [5]:
df_cpd_med_scores.shape

(1330, 7)

In [6]:
df_level4.shape

(27837, 984)

In [7]:
def get_cpds_replicates(df, df_lvl4):
    """
    This function returns all replicates id/names found in each compound 
    and in all doses(1-6)
    """
    
    dose_list = list(set(df_lvl4['dose'].unique().tolist()))[1:7]
    replicates_in_all = []
    cpds_replicates = {}
    for dose in dose_list:
        rep_list = []
        df_doses = df_lvl4[df_lvl4['dose'] == dose].copy()
        for cpd in df.index:
            replicate_names = df_doses[df_doses['pert_iname'] == cpd]['replicate_id'].values.tolist()
            rep_list += replicate_names
            if cpd not in cpds_replicates:
                cpds_replicates[cpd] = [replicate_names]
            else:
                cpds_replicates[cpd] += [replicate_names]
        replicates_in_all.append(rep_list)
        
    return replicates_in_all, cpds_replicates

In [8]:
replicates_in_all, cpds_replicates = get_cpds_replicates(df_cpd_med_scores, df_level4)

In [9]:
def replicates_per_cpd_size(df, df_lvl4, cpds_replicates):
    
    """
    This function gets all replicates ids for each distinct 
    cpd_size (i.e. number of replicates per cpd) classes
    
    Returns cpd_size_dict dictionary, with cpd_size as the keys, 
    and replicate_ids for each cpd_size as the values
    """
    
    df['replicate_id'] = list(cpds_replicates.values())
    dose_list = list(set(df_lvl4['dose'].unique().tolist()))[1:7]
    cpd_size_dict = {}
    for dose in dose_list:
        for size in df['cpd_size'].unique():
            rep_lists = []
            for idx in range(df[df['cpd_size'] == size].shape[0]):
                rep_ids = df[df['cpd_size'] == size]['replicate_id'].values.tolist()[idx][dose-1]
                rep_lists += rep_ids
            if size not in cpd_size_dict:
                cpd_size_dict[size] = [rep_lists]
            else:
                cpd_size_dict[size] += [rep_lists]
                
    return cpd_size_dict

In [10]:
cpd_size_dict = replicates_per_cpd_size(df_cpd_med_scores, df_level4, cpds_replicates)

In [11]:
def check_similar_replicates(replicates, dose, cpd_dict):
    """This function checks if two replicates are of the same compounds"""
    
    for x in range(len(replicates)):
        for y in range(x+1, len(replicates)):
            for kys in cpd_dict:
                if all(i in cpd_dict[kys][dose-1] for i in [replicates[x], replicates[y]]):
                    return True
    return False

In [12]:
def get_random_replicates(all_replicates, cpd_size, dose, replicates_ids, cpd_replicate_dict):
    """
    This function return a list of random replicates that are not of the same compounds
    or found in the current cpd's size list
    """
    while (True):
        random_replicates = random.sample(all_replicates, cpd_size)
        if not (any(rep in replicates_ids for rep in random_replicates) & 
                (check_similar_replicates(random_replicates, dose, cpd_replicate_dict))):
            break
    return random_replicates

In [13]:
def get_null_distribution_replicates(cpd_size_dict, dose_list, replicates_lists, cpd_replicate_dict, rand_num = 1000):
    
    """
    This function returns a null distribution dictionary, with CPD_SIZEs as the keys and 
    1000 lists of randomly selected replicate combinations as the values
    for each cpd_size class per DOSE(1-6)
    """
    null_distribution_reps = {}
    for dose in dose_list:
        for size in cpd_size_dict:
            replicates_ids = cpd_size_dict[size][dose-1]
            replicate_list = []
            for idx in range(rand_num):
                start_again = True
                while (start_again):
                    rand_cpds = get_random_replicates(replicates_lists[dose-1], size, dose, 
                                                      replicates_ids, cpd_replicate_dict)
                    if rand_cpds not in replicate_list:
                        start_again = False
                replicate_list.append(rand_cpds)
            if size not in null_distribution_reps:
                null_distribution_reps[size] = [replicate_list]
            else:
                null_distribution_reps[size] += [replicate_list]
    
    return null_distribution_reps

In [14]:
dose_list = list(set(df_level4['dose'].unique().tolist()))[1:7]
null_distribution_replicates = get_null_distribution_replicates(cpd_size_dict, dose_list, replicates_in_all, cpds_replicates)

In [15]:
def save_to_pickle(null_distribution, path, file_name):
    """This function saves the null distribution replicates ids into a pickle file"""
    
    if not os.path.exists(path):
        os.mkdir(path)
        
    with open(os.path.join(path, file_name), 'wb') as handle:
        pickle.dump(null_distribution, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
save_to_pickle(null_distribution_replicates, L1000_level4_path, 'null_distribution_W.pickle')

In [17]:
##load the null_distribution_moa from pickle
with open(os.path.join(L1000_level4_path, 'null_distribution_W.pickle'), 'rb') as handle:
    null_distribution_replicates = pickle.load(handle)

In [18]:
def assert_null_distribution(null_distribution_reps, dose_list):
    
    """
    This function assert that each of the list in the 1000 lists of 
    random replicate combination (per dose) for each cpd size are distinct with no duplicates
    """
    
    duplicates_reps = {}
    for dose in dose_list:
        for keys in null_distribution_reps:
            null_dist = null_distribution_reps[keys][dose-1]
            for reps in null_dist:
                dup_reps = []
                new_list = list(filter(lambda x: x != reps, null_dist))
                if (len(new_list) != len(null_dist) - 1):
                    dup_reps.append(reps)
            if dup_reps:
                if keys not in duplicates_reps:
                    duplicates_reps[keys] = [dup_reps]
                else:
                    duplicates_reps[keys] += [dup_reps]
    return duplicates_reps

In [19]:
duplicate_replicates = assert_null_distribution(null_distribution_replicates, dose_list)

In [20]:
duplicate_replicates ##no duplicates

{}

In [21]:
def calc_null_dist_median_scores(df, dose_num, replicate_lists):
    """
    This function calculate the median of the correlation 
    values for each list in the 1000 lists of random replicate 
    combination for each cpd size per dose
    """
    df_dose = df[df['dose'] == dose_num].copy()
    df_dose = df_dose.set_index('replicate_id').rename_axis(None, axis=0)
    df_dose.drop(['Metadata_broad_sample', 'pert_id', 'dose', 'pert_idose', 
                  'pert_iname', 'moa', 'sig_id'], axis = 1, inplace = True)
    median_corr_list = []
    for rep_list in replicate_lists:
        df_reps = df_dose.loc[rep_list].copy()
        reps_corr = df_reps.astype('float64').T.corr(method = 'spearman').values
        median_corr_val = median(list(reps_corr[np.triu_indices(len(reps_corr), k = 1)]))
        median_corr_list.append(median_corr_val)
    return median_corr_list

In [22]:
def get_null_dist_median_scores(null_distribution_cpds, dose_list, df):
    """ 
    This function calculate the median correlation scores for all 
    1000 lists of randomly combined compounds for each cpd_size class 
    across all doses (1-6)
    """
    null_distribution_medians = {}
    for key in null_distribution_cpds:
        median_score_list = []
        for dose in dose_list:
            replicate_median_scores = calc_null_dist_median_scores(df, dose, null_distribution_cpds[key][dose-1])
            median_score_list.append(replicate_median_scores)
        null_distribution_medians[key] = median_score_list
    return null_distribution_medians

In [23]:
null_distribution_medians = get_null_dist_median_scores(null_distribution_replicates, dose_list, df_level4)

**A P value can be computed nonparametrically by evaluating the probability of random replicates of different compounds having median similarity value greater than replicates of the same compounds.**

In [24]:
def get_p_value(median_scores_list, df, dose_name, cpd_name):
    """
    This function calculate the p-value from the 
    null_distribution median scores for each compound
    """
    actual_med = df.loc[cpd_name, dose_name]
    p_value = np.sum(median_scores_list >= actual_med) / len(median_scores_list)
    return p_value

In [25]:
def get_moa_p_vals(null_dist_median, dose_list, df_med_values):
    """
    This function returns a dict, with compounds as the keys and the compound's 
    p-values for each dose (1-6) as the values
    """
    null_p_vals = {}
    for key in null_dist_median:
        df_cpd_size = df_med_values[df_med_values['cpd_size'] == key]
        for cpd in df_cpd_size.index:
            dose_p_values = []
            for num in dose_list:
                dose_name = 'dose_' + str(num)
                cpd_p_value = get_p_value(null_dist_median[key][num-1], df_cpd_size, dose_name, cpd)
                dose_p_values.append(cpd_p_value)
            null_p_vals[cpd] = dose_p_values
    sorted_null_p_vals = {key:value for key, value in sorted(null_p_vals.items(), key=lambda item: item[0])}
    return sorted_null_p_vals

In [26]:
null_p_vals = get_moa_p_vals(null_distribution_medians, dose_list, df_cpd_med_scores)

In [27]:
df_null_p_vals = pd.DataFrame.from_dict(null_p_vals, orient='index', columns = ['dose_' + str(x) for x in dose_list])

In [28]:
df_null_p_vals['cpd_size'] = df_cpd_med_scores['cpd_size']

In [29]:
df_null_p_vals.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6,cpd_size
17-hydroxyprogesterone-caproate,0.935,0.105,0.045,0.001,0.013,0.140,3
2-iminobiotin,0.877,0.012,0.555,0.751,0.099,0.584,2
3-amino-benzamide,0.014,0.339,0.393,0.509,0.186,0.011,3
3-deazaadenosine,0.207,0.522,0.341,0.250,0.510,0.288,2
ABT-737,0.143,0.891,0.110,0.412,0.060,0.003,3
AICA-ribonucleotide,0.153,0.045,0.347,0.011,0.331,0.033,3
AKT-inhibitor-1-2,0.573,0.039,0.109,0.079,0.501,0.553,3
ALX-5407,0.030,0.098,0.002,0.081,0.120,0.013,3
AS-605240,0.980,0.029,0.848,0.808,0.027,0.066,2
AT-7519,0.005,0.187,0.044,0.000,0.000,0.000,3


In [30]:
def save_to_csv(df, path, file_name):
    """saves dataframes to csv"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    df.to_csv(os.path.join(path, file_name), index = False)

In [31]:
save_to_csv(df_null_p_vals.reset_index().rename({'index':'cpd'}, axis = 1), L1000_level4_path, 
            'cpd_replicate_p_values_W.csv')